# [Advent of Code 2022](https://adventofcode.com/2022)
<div align="right"><i>Ben Emery<br>December 2022</i></div>


## The toolbox

Generalised pieces of code that either can be used in multiple questions or that simply makes understand the implementation easier.

In [1]:
from collections import defaultdict
from functools import reduce
from heapq import heappush, heappop
import operator
import re


def Input(day, parser=str.strip, whole_file=False):
    "Fetch the data input from disk."
    filename = f"../data/advent2022/input{day}.txt"
    with open(filename) as fin:
        if whole_file:
            return parser(fin)
        return mapt(parser, fin)


def mapt(fn, *args):
    "Do a map, and convert the results to a tuple"
    return tuple(map(fn, *args))


class Node:
    def __init__(self, name, data=None):
        self.name = name
        self.data = data
        self._children = []
        self.parent = None

    def __str__(self):
        return f"<Node name={self.name} parent={self.parent.name if self.parent else None} data={self.data}>"

    def add_child(self, node):
        self._children.append(node)
        node.parent = self

    @property
    def children(self):
        return self._children[:]

    @property
    def root(self):
        if self.parent:
            return self.parent.root
        return self


def visit_post_order(node: Node):
    for child in node.children:
        yield from visit_post_order(child)
    yield node
    

def a_star(start, h_func, moves, cost=lambda s1, s2: 1):
    """A* implementation.

    Finds the shortest sequence of states from to a goal (a state where
    the hueristic function - h_func - is zero). We use a heap
    as our priority queue, and processes those with the smallest
    overall cost first (the cost of the path + the distance to target).

    start:  the initial state to explore from
    h_func: hueristic function that gives a "distance" to to target state,
            when this is zero we're done.
    moves:  function that generates all possible states from the supplied state
            (these can go bakckwards, but will never be processed)
    cost:   the cost of moving from ones state to another

    return: list of states used to find the final state, wll raise an exception
            if none was found.

    """
    # The priority queue that we'll be reading from
    queue = []
    # We often care about the path taken, so persist the lowest costing path
    # to each state
    previous = {start: None}
    # Lookup of state costs, we initialize at zero for the starting state
    costs = {start: 0}

    # Initialize our queue, this is ordered by path cost (f(n) = g(n) + h(n))
    add_to_queue = lambda state: heappush(queue, (costs[state] + h_func(state), state))

    # Recursively walk backwards to build the full path
    get_path = (
        lambda state: [] if state is None else get_path(previous[state]) + [state]
    )

    # Set the intial position and go!
    add_to_queue(start)

    while queue:
        _, state = heappop(queue)
        if h_func(state) == 0:
            # We're done!
            return get_path(state)

        for new_state in moves(state):
            new_cost = costs[state] + cost(state, new_state)

            if new_state not in costs or new_cost < costs[new_state]:
                # We've found a new state or a better path
                costs[new_state] = new_cost
                previous[new_state] = state
                # We've modified our costs in some way, we need
                # to explore from this state so add to the heap
                add_to_queue(new_state)

    # No solution was found
    raise Exception("No solution for A* was discovered.")

    
NEIGHBOUR4_DELTAS = (
              ( 0, -1),
    (-1,  0),           (1,  0),
              ( 0,  1),
)


def neighbours4(x, y):
    return tuple((x + dx, y + dy) for dx, dy in NEIGHBOUR4_DELTAS)

## [Day 1](https://adventofcode.com/2022/day/1)

Nothing too difficult for day 1, other than remembering how jupyter notebooks / python works...

### Part 1

In [2]:
data = Input(1, lambda s: int(s) if s.strip() else None)

def total_calories(data):
    elves = [0]
    for food in data:
        if food is None:
            elves.append(0)
            continue
        elves[-1] += food
    return elves

elves = total_calories(data)

max(elves)

70296

In [3]:
assert _ == 70296, "Day 1.1"

### Part 2

In [4]:
elves.sort()
sum(elves[-3:])

205381

In [5]:
assert _ == 205381, "Day 1.2"

## [Day 2](https://adventofcode.com/2022/day/2)

Again we're being eased in quite gently, unless you get dicts the wrong way around of course..

### Part 1

In [6]:
data = Input(2, lambda s: s.strip().split(" "))

ROCK = "R"
PAPER = "P"
SCISSORS = "S"

SCORES = {ROCK: 1, PAPER: 2, SCISSORS: 3}
WINNING = {SCISSORS: ROCK, ROCK: PAPER, PAPER: SCISSORS}
LOSING = dict((v, k) for k, v in WINNING.items())


def map_hands(player_1, player_2):
    map_1 = {"A": ROCK, "B": PAPER, "C": SCISSORS}
    map_2 = {"X": ROCK, "Y": PAPER, "Z": SCISSORS}
    return map_1[player_1], map_2[player_2]


def score_game(p1, p2):
    score = 3 if p1 == p2 else 0
    if p2 == WINNING[p1]:
        score = 6
    return score + SCORES[p2]


def score_all_games(data, hand_mapper):
    hands = (map_hands(*d) for d in data)
    return sum(score_game(*h) for h in hands)


score_all_games(data, map_hands)

11449

In [7]:
assert _ == 11449, "Day 2.1"

### Part 2

In [8]:
def map_hands(player_1, player_2):
    map_1 = {"A": "R", "B": "P", "C": "S"}
    p1 = map_1[player_1]

    if player_2 == "X":
        p2 = LOSING[p1]
    elif player_2 == "Y":
        p2 = p1
    elif player_2 == "Z":
        p2 = WINNING[p1]

    return p1, p2


score_all_games(data, map_hands)

13187

In [9]:
assert _ == 13187, "Day 2.2"

## [Day 3](https://adventofcode.com/2022/day/3)

First time we need a set, membership tests are O(1) so they make sense to use here. Python makes it quite easy to extend behaviour for part 2, but I find I'm missing a more functional approach..

### Part 1

In [10]:
data = Input(3)


def split_halves(contents):
    mid = int(len(contents) / 2)
    return contents[:mid], contents[mid:]


def find_common(*chunks):
    common = set(chunks[0])
    for chunk in chunks[1:]:
        common &= set(chunk)
    return tuple(common)


def score(s):
    ordinal = ord(s)
    if ordinal > 96:
        score = ordinal - ord("a") + 1
    else:
        score = ordinal - ord("A") + 26 + 1
    return score


def solve(all_data, grouper):
    total = 0
    for chunks in grouper(all_data):
        common = find_common(*chunks)
        total += score(common[0])
    return total


solve(data, lambda lines: map(split_halves, lines))

7967

In [11]:
assert _ == 7967, "Day 3.1"

In [12]:
def group_threes(lines):
    for idx in range(0, len(lines) - 1, 3):
        yield lines[idx : idx + 3]


solve(data, lambda lines: group_threes(lines))

2716

In [13]:
assert _ == 2716, "Day 3.2"

## [Day 4](https://adventofcode.com/2022/day/4)


I quite enjoyed this one, refactoring the check for detecting supersets meant that I could solve the second part with little effort, which is useually the besy way to go!

In [14]:
def parse_line(line):
    m = re.match("(\d+)-(\d+),(\d+)-(\d+)", line)
    a, b, c, d = m.groups()
    return ((int(a), int(b)), (int(c), int(d)))


data = Input(4, parse_line)


def count_supersets(lines, is_within):
    return sum(is_within(p1, p2) or is_within(p2, p1) for p1, p2 in lines)


def contains_entirely(p1, p2):
    return p1[0] >= p2[0] and p1[1] <= p2[1]


count_supersets(data, contains_entirely)

453

In [15]:
assert _ == 453, "Day 4.1"

### Part 2

In [16]:
def contains_partially(p1, p2):
    return (p1[0] >= p2[0] and p1[0] <= p2[1]) or (p1[1] <= p2[1] and p1[1] >= p2[0])


count_supersets(data, contains_partially)

919

In [17]:
assert _ == 919, "Day 4.2"

## [Day 5](https://adventofcode.com/2022/day/5)

Today's felt like one of those puzzles where the hardest part was parsing the input! Reminding myself how python's regex matches works was fun though, and it's also a good lesson about something I've struggled with in the past: pragmatism. Parsing the data doesn't need to be dynamic, we can apply what we know from the inputs to help us out (that there are 9 stacks for example). In fancier language I'd say that we can use a heuristic to simplify things, but who needs that first thing in the morning?

### Part 1

In [18]:
def parse_data(lines):
    stacks = [[], [], [], [], [], [], [], [], []]
    instructions = []

    def add_to_stack(line):
        for match in re.finditer("[A-Z]", line):
            stack = match.start() // 4
            stacks[stack].insert(0, match.group())

    def add_instruction(line):
        move, pos_from, pos_to = mapt(int, re.findall("\d+", line))
        instructions.append((move, pos_from - 1, pos_to - 1))

    stacks_complete = False
    for line in lines:
        if not line.strip():
            stacks_complete = True
            continue

        if not stacks_complete:
            add_to_stack(line)
        else:
            add_instruction(line)

    return stacks, instructions


data = Input(5, parse_data, whole_file=True)


def single_mover(stacks, instruction):
    move, pos_from, pos_to = instruction
    for _ in range(move):
        val = stacks[pos_from].pop()
        stacks[pos_to].append(val)


def follow_instructions(stacks, instructions, mover):
    new_stacks = [s[:] for s in stacks]

    for instruction in instructions:
        mover(new_stacks, instruction)
    return new_stacks


def top_crates(stacks):
    return "".join(s[-1] for s in stacks)


stacks = follow_instructions(*data, single_mover)
top_crates(stacks)

'LBLVVTVLP'

In [19]:
assert _ == "LBLVVTVLP", "Day 5.1"

### Part 2

In [20]:
def bulk_mover(stacks, instruction):
    move, pos_from, pos_to = instruction
    stacks[pos_to].extend(stacks[pos_from][-move:])
    stacks[pos_from] = stacks[pos_from][:-move]


stacks = follow_instructions(*data, bulk_mover)
top_crates(stacks)

'TPFFBDRJD'

In [21]:
assert _ == "TPFFBDRJD", "Day 5.2"

## [Day 6](https://adventofcode.com/2022/day/6)

Still nothing supuer difficult about this problem, using sets to track uniqueness may be a bit much, but given the size of the data it still returns instantly so why not!

### Part 1

In [22]:
data = Input(6)[0]


def iter_chunks(data, n):
    for i in range(0, len(data) - n):
        yield i, data[i : i + n]


def count_chars_to_packet(data, length=4):
    for idx, chunk in iter_chunks(data, length):
        if len(set(chunk)) == length:
            break
    chars_processed = idx + length
    return chars_processed


count_chars_to_packet(data, 4)

1598

In [23]:
assert _ == 1598, "Day 6.1"

### Part 2

In [24]:
count_chars_to_packet(data, 14)

2414

In [25]:
assert _ == 2414, "Day 6.2"

## [Day 7](https://adventofcode.com/2022/day/7)

First tree based problem! Had a really annoying bug with files / directories that had the same name, so ended up overwriting parts of the tree. Blugh. Significant part of the work went into building the tree, I suppose I didn't really need to do that, just build the sizes as you go and don't duplicate contributions to the same parent...

But that's waaaay less fun!


### Part 1

In [26]:
data = Input(7)


def _group_commands(lines):
    """Convert lines into a command, arguments, and outputs"""
    _, current_command, *args = lines[0].split(" ")
    outputs = []

    for line in lines[1:]:
        if line.startswith("$"):
            yield (current_command, args), outputs
            _, current_command, *args = line.split(" ")
            outputs = []
        else:
            outputs.append(tuple(line.split(" ")))
    yield (current_command, args), outputs


def build_tree(output_lines):
    """Take input data and build a directory tree"""
    node_lookup = {}

    def dir_key(parent, dir_name):
        if not parent:
            return dir_name
        return dir_key(parent.parent, parent.name) + "/" + dir_name

    def cd(args, _, current_node, node_lookup):
        new_dir = args[0]
        if new_dir == "..":
            return current_node.parent

        key = dir_key(current_node, new_dir)
        if key not in node_lookup:
            node = Node(new_dir)
            if current_node:
                current_node.add_child(node)
            node_lookup[key] = node

        return node_lookup[key]

    def ls(args, cmd_outputs, current_node, node_lookup):
        for chunks in cmd_outputs:
            if chunks[0] == "dir":
                dirname = chunks[1]
                key = dir_key(current_directory, dirname)

                if key not in node_lookup:
                    node_lookup[key] = Node(dirname)
                    current_node.add_child(node_lookup[key])
            else:
                size, filename = chunks
                current_node.add_child(Node(filename, int(size)))
        return current_node

    # execute all the commands
    current_directory = None
    for (cmd, args), outputs in _group_commands(output_lines):
        if cmd == "cd":
            command = cd
        elif cmd == "ls":
            command = ls
        else:
            raise Error(f"Unknown command {cmd}")
        current_directory = command(args, outputs, current_directory, node_lookup)
    return current_directory.root


def get_sub_directory_sizes(tree):
    sizes = defaultdict(int)

    for node in visit_post_order(tree):
        if not node.parent:
            # is root, don't care
            continue

        if node.data is not None:
            # contribute directly to the parent directory's size
            sizes[node.parent] += node.data
        else:
            # we'll have visited all the children of sub-directories before,
            # so use the cached result
            sizes[node.parent] += sizes[node]

    return sizes


def sum_directories_below_limit(data):
    tree = build_tree(data)
    sizes = get_sub_directory_sizes(tree)

    total = 0
    for dirname, size in sizes.items():
        if size <= 100000:
            total += size
    return total


sum_directories_below_limit(data)

1297683

In [27]:
assert _ == 1297683, "Day 7.1"

### Part 2

Now all the hard work of the first section is done, the second was pretty simple.

In [28]:
def find_smallest_directory_to_delete(data):
    tree = build_tree(data)
    directory_sizes = get_sub_directory_sizes(tree)

    total_size = directory_sizes[tree]

    remaining_space = 70000000 - total_size
    required_space = 30000000

    # start with the root node
    smallest = tree
    for node, size in directory_sizes.items():
        if node.data is not None:
            # is a file, skip
            continue

        if remaining_space + size > required_space:
            if size < directory_sizes[smallest]:
                smallest = node

    return directory_sizes[smallest]


find_smallest_directory_to_delete(data)

5756764

In [29]:
assert _ == 5756764, "Day 7.2"

In [30]:
data = Input(8, lambda s: mapt(int, s.strip()))


def count_visible_tress(lines):
    width = len(lines[0])
    height = len(lines)

    seen_trees = set()

    # first pass, left to right and right to left
    for y in range(1, height - 1):
        left_min_tree = lines[y][0]
        right_min_tree = lines[y][width - 1]

        for x in range(1, width - 1):
            if lines[y][x] > left_min_tree:
                seen_trees.add((x, y))
                left_min_tree = lines[y][x]
            if lines[y][width - x - 1] > right_min_tree:
                seen_trees.add((width - x - 1, y))
                right_min_tree = lines[y][width - x - 1]

    # second pass, top to bottom and bottom to top
    for x in range(1, width - 1):
        top_min_tree = lines[0][x]
        bottom_min_tree = lines[height - 1][x]

        for y in range(1, height - 1):
            if lines[y][x] > top_min_tree:
                seen_trees.add((x, y))
                top_min_tree = lines[y][x]
            if lines[height - y - 1][x] > bottom_min_tree:
                seen_trees.add((x, height - y - 1))
                bottom_min_tree = lines[height - y - 1][x]

    perimiter = 2 * width + 2 * height - 4
    return perimiter + len(seen_trees)


count_visible_tress(data)

1719

In [31]:
assert _ == 1719

### Part 2

In [32]:
def get_scenic_score(lines):
    width = len(lines[0])
    height = len(lines)

    best_score = 0
    for y in range(height):
        for x in range(width):
            tree = lines[y][x]

            see_left = 0
            for xl in range(x - 1, -1, -1):
                if lines[y][xl] >= tree:
                    see_left += 1
                    break
                see_left += 1

            see_right = 0
            for xr in range(x + 1, width):
                if lines[y][xr] >= tree:
                    see_right += 1
                    break
                see_right += 1

            see_top = 0
            for yt in range(y - 1, -1, -1):
                if lines[yt][x] >= tree:
                    see_top += 1
                    break
                see_top += 1

            see_bottom = 0
            for yb in range(y + 1, height):
                if lines[yb][x] >= tree:
                    see_bottom += 1
                    break
                see_bottom += 1

            scenic_score = see_left * see_top * see_right * see_bottom
            best_score = max(best_score, scenic_score)

    return best_score


get_scenic_score(data)

590824

In [33]:
assert _ == 590824, "Day 8.2"

## Day 9

This one killed me! I could not for the life of me conceptualise what the movements were doing. Definitely glad this one is over.

In [34]:
def parse_line(line):
    chunks = line.split(" ")
    return chunks[0], int(chunks[1])


def is_adjacent(p1, p2):
    return max(abs(a - b) for a, b in zip(p1, p2)) <= 1


def move_to_point(from_point, to_point):
    fx, fy = from_point
    tx, ty = to_point

    # if not on the same plane, then step in that direction
    move_point = lambda p1, p2: p1 if p1 == p2 else p1 + 1 if p1 < p2 else p1 - 1

    x = move_point(fx, tx)
    y = move_point(fy, ty)

    return x, y


def follow_motions(motions, rope_length=2):
    ropes = [(0, 0) for _ in range(rope_length)]

    deltas = {"R": (1, 0), "D": (0, -1), "L": (-1, 0), "U": (0, 1)}

    add_points = lambda p1, p2: tuple(a + b for a, b in zip(p1, p2))

    def step(direction, ropes):
        head_pos = ropes[0]
        dx, dy = deltas[direction]
        x, y = head_pos
        new_head_pos = (x + dx, y + dy)

        new_positions = [new_head_pos]
        for tail in ropes[1:]:
            target = new_positions[-1]
            if is_adjacent(tail, target):
                # within distance, no need to move
                new_positions.append(tail)
                continue

            new_tail_pos = move_to_point(tail, target)
            new_positions.append(new_tail_pos)
        return new_positions

    for direction, amount in motions:
        for _ in range(amount):
            ropes = step(direction, ropes)
            yield ropes


def count_unique_locations(motions, rope_length=2):
    visited = set()
    for rope in follow_motions(motions, rope_length):
        *_, tail_pos = rope
        visited.add(tail_pos)
    return len(visited)


data = Input(9, parse_line)
count_unique_locations(data)

6357

In [35]:
assert _ == 6357, "Day 9.1"

In [36]:
count_unique_locations(data, rope_length=10)

2627

In [37]:
assert _ == 2627, "Day 9.2"

## Day 10

This was quite fun, making use of python's generators to suspend execution meant that I could simulate the clock cycles quite nicely.

In [38]:
def parse_line(line):
    cmd, *args = line.strip().split(' ')
    if cmd == "addx":
        return cmd, int(args[0])
    return cmd, args

instructions = Input(10, parse_line)

def perform_instructions(instructions):
    # init the "reigister"
    x = 1
    for ins, *args in instructions:
        if ins == "noop":
            yield x
        elif ins == "addx":
            value = args[0]
            yield x
            x += value
            yield x

            
def sum_signal_strength(instructions):
    total = 0
    for executed_instruction_idx, x in enumerate(perform_instructions(instructions), 1):
        current_cycle = executed_instruction_idx + 1
        if (current_cycle - 20) % 40 == 0:
            total += current_cycle * x
    return total

            
sum_signal_strength(instructions)

13760

In [39]:
assert _ == 13760, "Day 10.1"

### Part 2

It's always quite fun when there's some ascii art invovled :D 

In [40]:
def draw_sprite(instructions):
    # not in the puzzle, but easier to read
    OFF_CHAR = " "
    ON_CHAR = "#"
    WIDTH = 40
    HEIGHT = 6
    rows = [[OFF_CHAR] * WIDTH for _ in range(HEIGHT)]

    for executed_instruction_idx, x in enumerate(perform_instructions(instructions), 1):
        row_being_drawn = executed_instruction_idx // WIDTH
        pixel_being_drawn = executed_instruction_idx % WIDTH

        if x >= pixel_being_drawn - 1 and x <= pixel_being_drawn + 1:
            rows[row_being_drawn][pixel_being_drawn] = ON_CHAR

    print("\n".join("".join(r) for r in rows))


draw_sprite(instructions)

 ##  #### #  # ####  ##  ###  #### #### 
#  # #    # #     # #  # #  # #    #    
#  # ###  ##     #  #    #  # ###  ###  
###  #    # #   #   #    ###  #    #    
# #  #    # #  #    #  # #    #    #    
#  # #    #  # ####  ##  #    #### #    


## Day 11

Well this one was interesting, I've been bitten by the Chinese Remainder Theorem before, so spent a bit of time in the write up for part 2 below.

There's some abose of python's `operator` library below which I'm not super happy about, but it'll do.

In [41]:
def parse_input(lines):
    monkeys = []
    ops = {"+": operator.add, "*": operator.mul}

    def get_new_func(operation, operation_value):
        if operation_value == "old":
            return lambda val: ops[operation](val, val)
        return lambda val: ops[operation](val, int(operation_value))

    # we're given a file input to parse, read all the lines and clear new line chars
    lines = [l.strip() for l in lines]
    for idx in range(0, len(lines), 7):
        starting_items = [int(d) for d in re.findall("\d+", lines[idx + 1])]

        *_, operation, operation_new = lines[idx + 2].split(" ")

        test_modulo = int(lines[idx + 3].split(" ")[-1])
        test_true_result = int(lines[idx + 4].split(" ")[-1])
        test_false_result = int(lines[idx + 5].split(" ")[-1])

        monkeys.append(
            {
                "starting_items": starting_items,
                "get_new_worry": get_new_func(operation, operation_new),
                "test_modulo": test_modulo,
                "true_monkey": test_true_result,
                "false_monkey": test_false_result,
            }
        )

    return monkeys


data = Input(11, parse_input, whole_file=True)


def fast_monkey_business(monkeys, round_count=20, apply_reduction=True):
    # clone the starting items as we mutate them in place
    starting_items = dict(
        (idx, m["starting_items"][:]) for idx, m in enumerate(monkeys)
    )
    counter = dict((idx, 0) for idx in range(len(monkeys)))

    # we know that the modulos are coprime, so we can safely reduce large modulos by their
    # lowest multiple, see text below for explanation
    module_multiple = reduce(operator.mul, [m["test_modulo"] for m in monkeys])
    # crt == chinese remainder theorem
    apply_crt = lambda worry: worry % module_multiple

    reduce_worry = apply_crt

    if apply_reduction:
        reduce_worry = lambda worry: worry // 3

    for _ in range(round_count):
        for idx, monkey in enumerate(monkeys):
            while starting_items[idx]:
                item = starting_items[idx].pop(0)
                new_worry = monkey["get_new_worry"](item)
                new_level = reduce_worry(new_worry)

                if new_level % monkey["test_modulo"] == 0:
                    new_target = monkey["true_monkey"]
                else:
                    new_target = monkey["false_monkey"]
                starting_items[new_target].append(new_level)

                counter[idx] += 1

    return counter


def find_busiest_monkeys(*args, **kwargs):
    counter = fast_monkey_business(*args, **kwargs)
    a, b = sorted(counter.values())[-2:]
    return a * b


find_busiest_monkeys(data)

50616

In [42]:
assert _ == 50616, "Day 11.1"

### Part 2

We're warned that the brute force of the first part won't work for the second, as the step of divising by 3 on line 47 is going away, and need to do something else.

Looking at the Test divisors from the input (5, 2, 13, 19, 11, 3, 7, 17), we can see that they are all prime, and thus have a greatest common divisor (GCD) of 1.

This means we can use the [Chinese Remainder Theorm](https://en.wikipedia.org/wiki/Chinese_remainder_theorem). 

...but what does that _mean_.

We know we need to perform several rounds of `X mod(y)`, but what happens if `X` is much, much larger than `y`? The operation becomes really really slow, it'd be nice if we could use a larger number to mod it against. As were dealing with modulo arithmitic, it's worth reminding oursleves (as I had to..) that `X mod(y)` is the same as `X mod(y * C) mod(y)` for _any_ C. i.e.

```
1345 mod(19) = 15
1345 mod(19 * 2) mod(19) = 15 mod (19) = 15
1345 mod(19 * 3) mod(19) = 34 mod (19) = 15
```

As we know that all our modulos do _not share a common factor_ then their multiple will be a large number that is the lowest common multiple of all of them. So taking the first three examples I've shown above:

```
1345 mod(2) = 1
1345 mod(2 * 3 * 19) mod(2) = 91 mod(2) = 1

1345 mod(3) = 1
1345 mod(2 * 3 * 19) mod(4) = 91 mod(3) = 1

1345 mod(19) = 15
1345 mod(2 * 3 * 19) mod(19) = 91 mod(15) = 15

```

Hopefully that's a useful example of how the CRT works.

In [43]:
# I just refactored the code in part one..
find_busiest_monkeys(data, 10_000, False)

11309046332

In [44]:
assert _ == 11309046332, "Day 11.2"

## Day 12

First breadth first search! I've just cracked out an A* implementation I wrote a few years ago, but it's always such a joy to use. A* is such an elegant algorithm, I love it!

For some extra fun, someone made a simulation of their solution in [minecraft](https://www.reddit.com/r/adventofcode/comments/zjsgaa/2022_day_12_part_2_in_minecraft/).

In [45]:
def find_all_cells(cell, grid):
    for y, row in enumerate(grid):
        try:
            x = row.index(cell)
            yield x, y
        except ValueError:
            pass


def find_cell(cell, grid):
    return next(find_all_cells(cell, grid))


def find_destination_position(grid):
    END_CELL = "E"
    return find_cell(END_CELL, grid)


def find_start_position(grid):
    START_CELL = "S"
    return find_cell(START_CELL, grid)


def fewest_steps(grid, start, destination):
    def normalise_grid(grid):
        rows = []
        for row in grid:
            new_row = []
            for cell in row:
                if cell == "S":
                    new_row.append(0)
                    continue
                if cell == "E":
                    new_row.append(ord("z") - ord("a"))
                    continue
                new_row.append(ord(cell) - ord("a"))
            rows.append(new_row)
        return rows

    grid_as_numbers = normalise_grid(grid)

    Y_LIMIT = len(grid) - 1
    X_LIMIT = len(grid[0]) - 1

    def h_func(state):
        """Cost function is manhatten distance from destination."""
        return abs(state[0] - destination[0]) + abs(state[1] - destination[1])

    def moves(state):
        x, y = state
        current_cell = grid_as_numbers[y][x]

        for (
            nx,
            ny,
        ) in neighbours4(x, y):
            if nx < 0 or ny < 0 or nx > X_LIMIT or ny > Y_LIMIT:
                continue

            new_cell = grid_as_numbers[ny][nx]
            if new_cell - current_cell > 1:
                # can't climb UP more than one, but can go down
                continue

            yield nx, ny

    shortest_path = a_star(start, h_func, moves)
    return len(shortest_path) - 1


def fewest_steps_fixed_start(grid):
    start = find_start_position(grid)
    destination = find_destination_position(grid)
    return fewest_steps(grid, start, destination)


data = Input(12)
fewest_steps_fixed_start(data)

423

In [46]:
assert _ == 423, "Day 12.1"

### Part 2

In [47]:
def fewest_steps_lowest_start(grid):
    destination = find_destination_position(grid)
    start_positions = find_all_cells("a", grid)
    return min(fewest_steps(grid, start, destination) for start in start_positions)


fewest_steps_lowest_start(data)

416

In [48]:
assert _ == 416, "Day 12.2"